In [2]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 67.9 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15637 sha256=6ba510927e0d7e6a832b6ffb67cdc40024eddd66f8c43e9008773792542eb088
  Stored in directory: /home/479e556e-3ace-47a4-9044-6fd66cff6268/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
Successfully built multitasking
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/6 [peewee]  WARNING: The script pwiz is installed in '/home/479e556e-3ace-47a4-9044-6fd66cff6268/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!pip install pandas_datareader

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import plotly.offline as pyo
import plotly.express as px
import pandas_datareader.data as reader

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'

In [29]:
df = yf.download('FDGRX', start='2015-01-01', end='2020-12-31')
df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,FDGRX,FDGRX,FDGRX,FDGRX,FDGRX
Date,,,,,
2015-01-02,3.295303,3.295303,3.295303,3.295303,0
2015-01-05,3.247788,3.247788,3.247788,3.247788,0
2015-01-06,3.208275,3.208275,3.208275,3.208275,0
2015-01-07,3.260291,3.260291,3.260291,3.260291,0
2015-01-08,3.318560,3.318560,3.318560,3.318560,0
...,...,...,...,...,...
2020-12-23,15.014370,15.014370,15.014370,15.014370,0
2020-12-24,15.014370,15.014370,15.014370,15.014370,0


In [30]:
log_returns = np.log(df.Close/df.Close.shift(1)).dropna()

In [35]:
monthly_cum_returns = np.exp(log_returns.resample('ME').sum()) - 1
monthly_cum_returns = monthly_cum_returns
monthly_cum_returns

Ticker,FDGRX
Date,
2015-01-31,-0.002431
2015-02-28,0.068234
2015-03-31,-0.013432
2015-04-30,0.001521
2015-05-31,0.025020
...,...
2020-08-31,0.126136
2020-09-30,-0.033199
2020-10-31,-0.028562


![FamaFrench](https://t9x4g5q5.delivery.rocketcdn.me/wp-content/uploads/2024/12/Fama-French-Three-Factor-Model-1024x514.webp)

In [36]:
factors = reader.DataReader('F-F_Research_Data_Factors', 'famafrench', start='2015-01-01', end='2020-12-31')[0]
factors

/home/479e556e-3ace-47a4-9044-6fd66cff6268/.local/lib/python3.12/site-packages/pandas_datareader/famafrench.py:114: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.

/home/479e556e-3ace-47a4-9044-6fd66cff6268/.local/lib/python3.12/site-packages/pandas_datareader/famafrench.py:114: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.



,Mkt-RF,SMB,HML,RF
Date,,,,
2015-01,-3.10,-0.59,-3.45,0.00
2015-02,6.13,0.61,-1.79,0.00
2015-03,-1.11,3.05,-0.38,0.00
2015-04,0.59,-2.99,1.80,0.00
2015-05,1.37,0.95,-1.11,0.00
...,...,...,...,...
2020-08,7.62,-0.20,-3.02,0.01
2020-09,-3.64,0.06,-2.72,0.01
2020-10,-2.08,4.34,4.25,0.01


Note: we need to have same number of rows for both monthly_cum_returns and factors

In [37]:
monthly_cum_returns.index = factors.index

Note: We need to make index similar for both of them (month and factor)

In [38]:
monthly_cum_returns

Ticker,FDGRX
Date,
2015-01,-0.002431
2015-02,0.068234
2015-03,-0.013432
2015-04,0.001521
2015-05,0.025020
...,...
2020-08,0.126136
2020-09,-0.033199
2020-10,-0.028562


In [39]:
merge = pd.merge(monthly_cum_returns, factors, on='Date')
merge

,FDGRX,Mkt-RF,SMB,HML,RF
Date,,,,,
2015-01,-0.002431,-3.10,-0.59,-3.45,0.00
2015-02,0.068234,6.13,0.61,-1.79,0.00
2015-03,-0.013432,-1.11,3.05,-0.38,0.00
2015-04,0.001521,0.59,-2.99,1.80,0.00
2015-05,0.025020,1.37,0.95,-1.11,0.00
...,...,...,...,...,...
2020-08,0.126136,7.62,-0.20,-3.02,0.01
2020-09,-0.033199,-3.64,0.06,-2.72,0.01
2020-10,-0.028562,-2.08,4.34,4.25,0.01


Now that we merged the data perfectly, we need to make the dataset similar. Our monthly cum returns are in numbers while fama french data is in percentage. So we need to change fama french data set into numbers. For this we will divide it by 100

In [40]:
merge[['Mkt-RF','SMB','HML','RF']] = merge[['Mkt-RF','SMB','HML','RF']]/100
merge

,FDGRX,Mkt-RF,SMB,HML,RF
Date,,,,,
2015-01,-0.002431,-0.0310,-0.0059,-0.0345,0.0000
2015-02,0.068234,0.0613,0.0061,-0.0179,0.0000
2015-03,-0.013432,-0.0111,0.0305,-0.0038,0.0000
2015-04,0.001521,0.0059,-0.0299,0.0180,0.0000
2015-05,0.025020,0.0137,0.0095,-0.0111,0.0000
...,...,...,...,...,...
2020-08,0.126136,0.0762,-0.0020,-0.0302,0.0001
2020-09,-0.033199,-0.0364,0.0006,-0.0272,0.0001
2020-10,-0.028562,-0.0208,0.0434,0.0425,0.0001


Now we need to calculate Ri - Rf, Left side of equation

In [41]:
merge['FDGRX-RF'] = merge.FDGRX - merge.RF
merge

,FDGRX,Mkt-RF,SMB,HML,RF,FDGRX-RF
Date,,,,,,
2015-01,-0.002431,-0.0310,-0.0059,-0.0345,0.0000,-0.002431
2015-02,0.068234,0.0613,0.0061,-0.0179,0.0000,0.068234
2015-03,-0.013432,-0.0111,0.0305,-0.0038,0.0000,-0.013432
2015-04,0.001521,0.0059,-0.0299,0.0180,0.0000,0.001521
2015-05,0.025020,0.0137,0.0095,-0.0111,0.0000,0.025020
...,...,...,...,...,...,...
2020-08,0.126136,0.0762,-0.0020,-0.0302,0.0001,0.126036
2020-09,-0.033199,-0.0364,0.0006,-0.0272,0.0001,-0.033299
2020-10,-0.028562,-0.0208,0.0434,0.0425,0.0001,-0.028662


In [42]:
y = merge ['FDGRX-RF']
X = merge[['Mkt-RF','SMB','HML']]

X_sm = sm.add_constant(X)

In [43]:
model = sm.OLS(y,X_sm)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               FDGRX-RF   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.851
Method:                 Least Squares   F-statistic:                     135.9
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           1.15e-28
Time:                        09:59:25   Log-Likelihood:                 175.23
No. Observations:                  72   AIC:                            -342.5
Df Residuals:                      68   BIC:                            -333.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0073      0.003      2.642      0.010       0.002       0.013
Mkt-RF         1.1293      0.062     18.338      0.000       1.006       1.252
SMB            0.1956      0.110      1.778      0.080      -0.024       0.415
HML           -0.4678      0.087     -5.401      0.000      -0.641      -0.295
==============================================================================
Omnibus:                       25.581   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.781
Skew:                           1.376   Prob(JB):                     1.39e-09
Kurtosis:                       5.454   Cond. No.                         44.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""